# Make_indicator_dict

This notebook is used to make a dictionary that allows the lookup of full name, table of origin, and percentage of missing data for each column in the combined dataset. The columns have indicator codes that are difficult to understand, so this dictionary is a helpful resourse.

## Part 1: Match indicators with names

#### Import libraries

In [1]:
import pandas as pd

/opt/conda/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


#### Importing the original tables that still had the Indicator Name included then making a list of the tables. SPI is omitted as it was used in the original df creation.

In [2]:
pop=pd.read_csv("../initial_data/Population-EstimatesData.csv", sep=",")
spi=pd.read_csv("../initial_data/SPIData.csv", sep=",")
gen=pd.read_csv("../initial_data/Gender_StatsData.csv", sep=",")
hnp=pd.read_csv("../initial_data/HNP_StatsData.csv", sep=",")
wdi=pd.read_csv("../initial_data/WDIData.csv", sep=",")
tabs=['pop', 'gen', 'hnp', 'wdi']
    

#### Making a dataframe using the Indicator Name and Indicator Code columns from SPI called "Indicators"
#### Using a for loop to go through the list of tables, extracting the two columns we want, and appending them to the bottom of the Indicators df we're creating
#### Dropping duplicates and resetting the index on the new df we're making
#### Using a for loop that goes the length of the df and removes periods and underscores from the Indicator Code column
#### Writing the df to CSV

In [3]:
Indicators=spi[['Indicator Code', 'Indicator Name']]
for x in tabs:
    codes=eval(x)[['Indicator Code', 'Indicator Name']]
    Indicators=Indicators.append(codes)
Indicators=Indicators.drop_duplicates().reset_index(drop=True)
for row in range(len(Indicators['Indicator Code'])):
    Indicators['Indicator Code'][row]=Indicators['Indicator Code'][row].replace(".","").replace("_","")
Indicators.drop_duplicates(subset='Indicator Code').to_csv("Indicator_Dict.csv", index=False)

#### Checking and verifying 

In [4]:
check=pd.read_csv("Indicator_Dict.csv")
check.shape

(2265, 2)

In [5]:
check.drop_duplicates(subset='Indicator Code').shape

(2265, 2)

## Part 2: Make table consisting of indicator code, percentage of data missing, and table of origin

### Connect to DB

In [6]:
username = input('Enter username: ')

Enter username: bmgwd9


In [7]:
import getpass
import pandas as pd
mypasswd = getpass.getpass()
#username = input('Enter username')
host = 'pgsql.dsa.lan'
database = 'caponl_22g3'
# Then connects to the DB
from sqlalchemy.engine.url import URL
from sqlalchemy import create_engine

# SQLAlchemy Connection Parameters
postgres_db = {'drivername': 'postgres',
               'username': username,
               'password': mypasswd,
               'host': host,
               'database' :database}
engine = create_engine(URL(**postgres_db), echo=True)
connection_string = f"postgres://{username}:{mypasswd}@{host}/{database}"
%load_ext sql
%sql $connection_string 
del mypasswd

········


### Use a for loop on a list of tables to iterate through the following process:
#### Record the name of the current table in the 'tab' variable
#### Save a SQL statement with the "table" to be designated in a format statement
#### Use the exec function to assign a SQL table to the variable
#### Within the table, iterate through the columns
#### In each column, calculate the percentage of not nulls and append that to the tallies df
#### Remove all records from the tallies df for index, Country Name, Country Code, or variable indicators (not actual indicators)
#### Export to CSV

In [8]:
tallies=pd.DataFrame(columns=['Table', 'Indicator', 'PCT'])
tables=["WB_SPI", "WB_Gender", "WB_HNP", "WB_Pop", "WB_WDI"]
for v in tables:
    tab=str(v)
    sql='Select * from {};'
    exec("v=pd.read_sql(sql.format(v), engine)")
    for x in v.columns:
        tallies=tallies.append(pd.DataFrame([[tab, x, len(v[-pd.isnull(v[x])])/len(v)]], columns=['Table', 'Indicator', 'PCT']))
tallies=tallies[(tallies['Indicator']!='index') & (tallies['Indicator']!='Country Name') & (tallies['Indicator']!='Country Code') & (tallies['Indicator']!='variable')]
tallies.to_csv("WB_results.csv", sep=",", index=False)


2022-04-19 16:42:11,306 INFO sqlalchemy.engine.base.Engine select version()
2022-04-19 16:42:11,307 INFO sqlalchemy.engine.base.Engine {}
2022-04-19 16:42:11,308 INFO sqlalchemy.engine.base.Engine select current_schema()
2022-04-19 16:42:11,309 INFO sqlalchemy.engine.base.Engine {}
2022-04-19 16:42:11,310 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2022-04-19 16:42:11,311 INFO sqlalchemy.engine.base.Engine {}
2022-04-19 16:42:11,312 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2022-04-19 16:42:11,313 INFO sqlalchemy.engine.base.Engine {}
2022-04-19 16:42:11,314 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2022-04-19 16:42:11,315 INFO sqlalchemy.engine.base.Engine {}
2022-04-19 16:42:11,316 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
20

#### Reading it back in to verify and check shape

In [9]:
pcts=pd.read_csv('WB_results.csv')
print(pcts.shape)
pcts.drop_duplicates(subset='Indicator').shape

(2189, 3)


(2189, 3)

## Part 3: Combine the two tables we made

In [10]:
merged=check.merge(pcts, left_on='Indicator Code', right_on='Indicator', how='inner')

In [11]:
merged.head()

,Indicator Code,Indicator Name,Table,Indicator,PCT
0,SPID412AGRI,Agriculture census (Availability score over 20...,WB_SPI,SPID412AGRI,0.247984
1,SPID415AGSVY,Agriculture survey (Availability score over 10...,WB_SPI,SPID415AGSVY,0.875576
2,SPID15POV,Availability of Comparable Poverty headcount r...,WB_SPI,SPID15POV,1.000000
3,SPID15CHLDMORT,"Availability of Mortality rate, under-5 (per 1...",WB_SPI,SPID15CHLDMORT,1.000000
4,SPID5210GSBP,Business process,WB_SPI,SPID5210GSBP,0.339862


In [12]:
merged.to_csv("merged_dict.csv")

In [13]:
for x in merged[merged['PCT']>.7]['Indicator Name']:
    print(x)

Agriculture survey (Availability score over 10 years)
Availability of Comparable Poverty headcount ratio at $1.90 a day (5 year moving average)
Availability of Mortality rate, under-5 (per 1,000 live births) data meeting quality standards according to UN IGME  (5 year moving average)
Business/establishment census (Availability score over 20 years)
Business/establishment survey (Availability score over 10 years)
Dimension 1.5: Data use by international organizations
Dimension 3.1: Social Statistics
Dimension 3.2: Economic Statistics
Dimension 3.3: Environmental Statistics
Dimension 3.4: Institutional Statistics
Dimension 4.1: Censuses and Surveys - Surveys only
GOAL 1: No Poverty (5 year moving average)
GOAL 10: Reduced Inequality (5 year moving average)
GOAL 11: Sustainable Cities and Communities (5 year moving average)
GOAL 12: Responsible Consumption and Production (5 year moving average)
GOAL 13: Climate Action (5 year moving average)
GOAL 15: Life on Land (5 year moving average)
GO

In [14]:
merged[merged['Indicator Name']=='Surface area (sq. km)']

,Indicator Code,Indicator Name,Table,Indicator,PCT
2090,AGSRFTOTLK2,Surface area (sq. km),WB_WDI,AGSRFTOTLK2,0.934736


## This concludes our milestone 1 work. Here is a link to milestone 2:

https://europa.dsa.missouri.edu/user/bmgwd9/notebooks/sp22Capstone_01_Group03/CapstoneOrganization/M2/M2.ipynb